In [ ]:
import os
from PIL import Image
import requests
import json
from PIL import Image
from io import BytesIO

def get():
    # Set your Unsplash API access key
    access_key = ""

    # Set the endpoint URL for the API request
    url = "https://api.unsplash.com/search/photos"

    # Set the parameters for the API request
    params = {
        "query": "mention_title_here", #lakes
        "orientation": "portrait",
        "per_page": 100,
    }
    

    # Add the access key to the API request headers
    headers = {
        "Authorization": f"Client-ID {access_key}"
    }

    # Send the API request
    response = requests.get(url, params=params, headers=headers)

    # Parse the API response as JSON
    data = json.loads(response.text)

    # Check if the response contains search results
    if "results" in data:
        # Loop through the search results and download the images
        for result in data["results"]:
            # Get the URL of the image from the search result
            image_url = result["urls"]["raw"]
            
            # Download the image
            response = requests.get(image_url)
            
            # Open the image using Pillow
            image = Image.open(BytesIO(response.content))
            
            # Rotate the image 90 degrees clockwise
            
            # Save the image to a file
            image.save("100Images/"+f"{result['id']}.jpg")
    else:
        # Check if there were no search results for the specified query
        if data["total"] == 0:
            print("No search results found for the specified query.")
        else:
            print("An error occurred while processing the API request.")


get()



In [ ]:
import os
from PIL import Image
from multiprocessing import Pool

def resize_and_paste_image(args):
    filename, num_cols, max_width, max_height = args
    im = Image.open(filename)
    # calculate the aspect ratio of the image
    aspect_ratio = im.size[0] / im.size[1]
    # calculate the new width and height of the image based on the max dimensions and aspect ratio
    if max_width / aspect_ratio < max_height:
        new_width = int(max_width)
        new_height = int(max_width / aspect_ratio)
    else:
        new_width = int(max_height * aspect_ratio)
        new_height = int(max_height)
    im = im.resize((new_width, new_height))
    return im

def create_collage(directory, num_rows, num_cols, output_filename):
    image_list = [os.path.join(directory, f) for f in os.listdir(directory)
                  if f.endswith('.jpg') or f.endswith('.png')]
    image_list=image_list[0:100]
    with Image.open(image_list[0]) as first_image:
        max_width, max_height = first_image.size

    args_list = [(filename, num_cols, max_width, max_height) for filename in image_list]
    with Pool() as p:
        resized_images = p.map(resize_and_paste_image, args_list)

    new_width = num_cols * max_width
    new_height = num_rows * max_height
    new_im = Image.new('RGB', (new_width, new_height))
    for i, im in enumerate(resized_images):
        col = i % num_cols
        row = i // num_cols
        x_offset = col * max_width + (max_width - im.size[0]) // 2
        y_offset = row * max_height + (max_height - im.size[1]) // 2
        new_im.paste(im, (x_offset, y_offset))

    new_im.save(output_filename)


create_collage('100Images', 10,10,'Output/Collage.jpg')
